In [1]:
import numpy as np
import pandas as pd

In [21]:
import pandas as pd
import numpy as np
import sklearn

import warnings
#warnings.filterwarnings("ignore")

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

N_FOLDS = 2

In [3]:
df_train  = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv')
#df_train
df_train['order_completed_at'] = pd.to_datetime(df_train['order_completed_at'])

In [4]:
df_test = pd.read_csv("./ngwl-predict-customer-churn/sample_submission.csv", sep=';')
df_test

,Id,Predicted
0,19843,NaN
1,471287,NaN
2,342522,NaN
3,457799,NaN
4,233778,NaN
...,...,...
214604,16841,NaN
214605,69348,NaN
214606,594974,NaN
214607,320024,NaN


In [5]:
df_train

,phone_id,order_completed_at,target
0,19843,2020-01-01,1.0
1,19843,2020-02-01,0.0
2,19843,2020-03-01,0.0
3,19843,2020-04-01,0.0
4,19843,2020-05-01,0.0
...,...,...,...
1159571,23853,2020-03-01,0.0
1159572,23853,2020-04-01,0.0
1159573,23853,2020-05-01,0.0
1159574,23853,2020-06-01,1.0


In [8]:
phone2month_activ = {}
for row_idx, row in df_train.iterrows():
    if row_idx % 40000 == 0:
        print(row_idx)
    if row['phone_id'] not in phone2month_activ:
        phone2month_activ[row['phone_id']] = {}
    phone2month_activ[row['phone_id']][row['order_completed_at'].month + 1] = row['target']
phone2month_activ

0
40000
80000
120000
160000
200000
240000
280000
320000
360000
400000
440000
480000
520000
560000
600000
640000
680000
720000
760000
800000
840000
880000
920000
960000
1000000
1040000
1080000
1120000


{19843: {2: 1.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 1.0},
 173074: {6: 0.0, 7: 0.0, 8: 0.0},
 101944: {2: 0.0, 3: 1.0, 4: 0.0, 5: 1.0, 6: 0.0, 7: 0.0, 8: 0.0},
 615032: {5: 1.0, 6: 0.0, 7: 0.0, 8: 0.0},
 342522: {2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 1.0, 8: 0.0},
 457799: {8: 1.0},
 702441: {5: 1.0, 6: 0.0, 7: 0.0, 8: 0.0},
 233778: {6: 1.0, 7: 0.0, 8: 1.0},
 40738: {3: 0.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 0.0},
 174054: {5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0},
 31838: {2: 0.0, 3: 1.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0},
 155699: {4: 0.0, 5: 0.0, 6: 1.0, 7: 1.0, 8: 1.0},
 57297: {8: 1.0},
 186723: {7: 1.0, 8: 1.0},
 37901: {5: 1.0, 6: 0.0, 7: 0.0, 8: 0.0},
 86829: {5: 1.0, 6: 1.0, 7: 0.0, 8: 0.0},
 68581: {6: 1.0, 7: 0.0, 8: 0.0},
 115937: {4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 0.0},
 50685: {6: 1.0, 7: 1.0, 8: 1.0},
 56463: {2: 0.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 0.0},
 478450: {4: 0.0, 5: 0.0, 6: 0.0, 7: 1.0, 8: 1.0},
 454226: {4: 1.0, 5: 0.0, 6: 0.0, 7: 0.0, 

In [22]:
train_data_def = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv', sep=',')
#print(train_data_def)

train_start = '2020-06'
test_start = '2020-07'

train_data = train_data_def[train_data_def['order_completed_at']==train_start]
print(train_data['phone_id'].nunique())
print(train_data['target'].value_counts())

test_data = train_data_def[train_data_def['order_completed_at']==test_start]
print(test_data['phone_id'].nunique())
print(test_data['target'].value_counts())

train_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(train_data))]
train_data['fold_num'].value_counts()
train_data['month']=6

test_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(test_data))]
test_data['fold_num'].value_counts()
test_data['month']=7


train_folds = []
test_folds = []

for i in range(1, N_FOLDS+1):
    local_data_train = train_data.loc[train_data['fold_num']==i]
    local_data_test = test_data.loc[test_data['fold_num']==i]

    del local_data_train['fold_num']
    del local_data_test['fold_num']
    
    del local_data_train['order_completed_at']
    del local_data_test['order_completed_at']
    
    train_folds.append(local_data_train.reset_index(drop=True))
    test_folds.append(local_data_test.reset_index(drop=True))

254285
0.0    127797
1.0    126488
Name: target, dtype: int64
284449
0.0    151807
1.0    132642
Name: target, dtype: int64


/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [ ]:
fea = pd.read_csv('fea/f_shipments.csv')

clf = SGDClassifier()

scores = []
for i in range(N_FOLDS):
    
    print(i)
    X_tr, y_tr = get_X_y(train_folds, i, fea)
    X_te, y_te = get_X_y(test_folds, i, fea)    
    
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)
    
    scores.append(f1_score(y_te, preds))
    break

In [8]:
client_features = pd.read_pickle("./fea_all_2_ship_plus_order_plus_phone.pkl")
client_features

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
0,1,98.0,98.0,98.0,98.0,0.000000,0.0,0.000000,0.000000,22024,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,366.0,158.0,208.0,183.0,35.355339,0.0,0.000000,0.000000,67946,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,233.0,233.0,233.0,233.0,0.000000,0.0,0.000000,0.000000,34195,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,248.0,248.0,248.0,248.0,0.000000,0.0,0.000000,0.000000,40777,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,31265,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155784,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,5320,...,1.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1155785,1,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,12270,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1155786,1,0.0,0.0,0.0,0.0,0.000000,5.0,-163.770004,-163.770004,14039,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1155787,1,0.0,0.0,0.0,0.0,0.000000,0.0,-193.270004,-193.270004,18470,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
client_features.set_index

In [23]:
client_features.columns

Index(['order_id_nunique', 'total_cost_sum', 'total_cost_min',
       'total_cost_max', 'total_cost_mean', 'total_cost_std', 'rate_mean',
       'promo_total_sum', 'promo_total_mean', 'total_weight_sum',
       'total_weight_mean', 'month', 'phone_id', 'month_after_first', 'age',
       'genger_nan', 'gender_male', 'gender_female', 'order_state_complete',
       'order_state_canceled', 'order_state_resumed', 'order_state_cart',
       'shipment_state_shipped', 'shipment_state_canceled',
       'shipment_state_collecting', 'shipment_state_ready',
       'shipment_state_shipping', 'shipment_state_ready_to_ship',
       'shipment_state_pending'],
      dtype='object')

In [69]:
len(set(df_test['Id'].values).difference(set(client_features[client_features['month'].isin([7, 8])]['phone_id'].values)))

2

In [66]:
client_features[client_features['month'].isin([6, 7])]

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
6,1,0.0,0.0,0.0,0.0,0.000000,5.0,-200.00,-200.00,28032,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2,356.0,158.0,198.0,178.0,28.284271,0.0,0.00,0.00,56717,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1,234.0,234.0,234.0,234.0,0.000000,0.0,0.00,0.00,46266,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
23,2,0.0,0.0,0.0,0.0,0.000000,0.0,-289.34,-144.67,17805,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
24,3,0.0,0.0,0.0,0.0,0.000000,0.0,-750.00,-250.00,27218,...,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154083,1,158.0,158.0,158.0,158.0,0.000000,5.0,0.00,0.00,3740,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1154087,1,0.0,0.0,0.0,0.0,0.000000,5.0,0.00,0.00,7305,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1154095,1,99.0,99.0,99.0,99.0,0.000000,4.0,0.00,0.00,3930,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1154111,1,99.0,99.0,99.0,99.0,0.000000,5.0,0.00,0.00,1864,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
client_features

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
0,1,98.0,98.0,98.0,98.0,0.000000,0.0,0.000000,0.000000,22024,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,366.0,158.0,208.0,183.0,35.355339,0.0,0.000000,0.000000,67946,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,233.0,233.0,233.0,233.0,0.000000,0.0,0.000000,0.000000,34195,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,248.0,248.0,248.0,248.0,0.000000,0.0,0.000000,0.000000,40777,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,31265,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155784,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,5320,...,1.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1155785,1,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,12270,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1155786,1,0.0,0.0,0.0,0.0,0.000000,5.0,-163.770004,-163.770004,14039,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1155787,1,0.0,0.0,0.0,0.0,0.000000,0.0,-193.270004,-193.270004,18470,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
client_features_6 = client_features[client_features['month'] == 6]
client_features_7 = client_features[client_features['month'] == 7]
client_features_56 = pd.merge(client_features[client_features['month'] == 5],
                              client_features[client_features['month'] == 6],
         how='outer', on='phone_id')
client_features_45 = pd.merge(client_features[client_features['month'] == 4],
                              client_features[client_features['month'] == 5],
                              how='outer', on='phone_id'
)   
client_features_67 = pd.merge(
    client_features[client_features['month'] == 6],
    client_features[client_features['month'] == 7],
    how='outer', on='phone_id'
)                          
client_features_678 = pd.merge(
    client_features_67,
    client_features[client_features['month'] == 8],
    how='outer', on='phone_id'
)
                              
client_features_567 = pd.merge(client_features_56, client_features[client_features['month'] == 7],
         how='outer', on='phone_id')
client_features_456 = pd.merge(client_features_45, client_features[client_features['month'] == 6],
         how='outer', on='phone_id')
client_features_567

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
0,1.0,248.0,248.0,248.0,248.0,0.000000,0.00,0.0,0.0,40777.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,168.0,168.0,168.0,168.0,0.000000,1.00,-250.0,-250.0,25106.0,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,331.0,163.0,168.0,165.5,3.535534,2.00,0.0,0.0,56605.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,316.0,0.0,158.0,79.0,91.221343,1.25,-600.0,-150.0,57316.0,...,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,158.0,158.0,158.0,158.0,0.000000,5.00,0.0,0.0,3990.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
client_features_456[['month_after_first',
 'month_after_first_x',
 'month_after_first_y']]

,month_after_first,month_after_first_x,month_after_first_y
0,NaN,4.0,NaN
1,NaN,2.0,NaN
2,1.0,2.0,3.0
3,3.0,4.0,5.0
4,NaN,0.0,1.0
...,...,...,...
370282,0.0,NaN,NaN
370283,0.0,NaN,NaN
370284,0.0,NaN,NaN
370285,0.0,NaN,NaN


In [148]:
sorted(client_features_456.columns)

['age',
 'age_x',
 'age_y',
 'gender_female',
 'gender_female_x',
 'gender_female_y',
 'gender_male',
 'gender_male_x',
 'gender_male_y',
 'genger_nan',
 'genger_nan_x',
 'genger_nan_y',
 'month',
 'month_after_first',
 'month_after_first_x',
 'month_after_first_y',
 'month_x',
 'month_y',
 'order_id_nunique',
 'order_id_nunique_x',
 'order_id_nunique_y',
 'order_state_canceled',
 'order_state_canceled_x',
 'order_state_canceled_y',
 'order_state_cart',
 'order_state_cart_x',
 'order_state_cart_y',
 'order_state_complete',
 'order_state_complete_x',
 'order_state_complete_y',
 'order_state_resumed',
 'order_state_resumed_x',
 'order_state_resumed_y',
 'phone_id',
 'promo_total_mean',
 'promo_total_mean_x',
 'promo_total_mean_y',
 'promo_total_sum',
 'promo_total_sum_x',
 'promo_total_sum_y',
 'rate_mean',
 'rate_mean_x',
 'rate_mean_y',
 'shipment_state_canceled',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_collecting',
 'shipment_state_collecting_x',
 

In [39]:
client_features_567['phone_id']

0              2
1              3
2              4
3              9
4             10
           ...  
404032    717314
404033    717399
404034    717442
404035    717445
404036    717450
Name: phone_id, Length: 404037, dtype: int64

In [40]:
client_features_567[client_features_567['phone_id'] == 717445]

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
404035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
 client_features[client_features['phone_id'] == 717445]['month']

1154082    7
Name: month, dtype: int64

In [38]:
len(set(df_test['Id'].unique()).difference(set(client_features_567['phone_id'].unique())))

34437

In [32]:
df_test

,Id,Predicted
0,19843,NaN
1,471287,NaN
2,342522,NaN
3,457799,NaN
4,233778,NaN
...,...,...
214604,16841,NaN
214605,69348,NaN
214606,594974,NaN
214607,320024,NaN


In [27]:
client_features[client_features['month'] == 5]

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
3,1,248.0,248.0,248.0,248.0,0.000000,0.00,0.0,0.0,40777,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,168.0,168.0,168.0,168.0,0.000000,1.00,-250.0,-250.0,25106,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2,331.0,163.0,168.0,165.5,3.535534,2.00,0.0,0.0,56605,...,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0
22,4,316.0,0.0,158.0,79.0,91.221343,1.25,-600.0,-150.0,57316,...,NaN,NaN,NaN,4.0,0.0,0.0,0.0,0.0,0.0,0.0
25,1,158.0,158.0,158.0,158.0,0.000000,5.00,0.0,0.0,3990,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154105,1,99.0,99.0,99.0,99.0,0.000000,0.00,0.0,0.0,10080,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1154109,1,98.0,98.0,98.0,98.0,0.000000,4.00,0.0,0.0,8555,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1154112,1,99.0,99.0,99.0,99.0,0.000000,5.00,0.0,0.0,27000,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1154113,1,213.0,213.0,213.0,213.0,0.000000,0.00,0.0,0.0,53000,...,NaN,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
features_3_month = pd.merge(left, right, how='inner', on=None, ([
        client_features[client_features['month'] == '5'],
        client_features[client_features['month'] == '6'],
        client_features[client_features['month'] == '7'],
        ], axis = 1
    )


In [52]:
client_features_567

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
0,1.0,248.0,248.0,248.0,248.0,0.000000,0.00,0.0,0.0,40777.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,168.0,168.0,168.0,168.0,0.000000,1.00,-250.0,-250.0,25106.0,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,331.0,163.0,168.0,165.5,3.535534,2.00,0.0,0.0,56605.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,316.0,0.0,158.0,79.0,91.221343,1.25,-600.0,-150.0,57316.0,...,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,158.0,158.0,158.0,158.0,0.000000,5.00,0.0,0.0,3990.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
client_features_567['phone_id'].nunique()

404037

In [85]:
client_features.columns

Index(['order_id_nunique', 'total_cost_sum', 'total_cost_min',
       'total_cost_max', 'total_cost_mean', 'total_cost_std', 'rate_mean',
       'promo_total_sum', 'promo_total_mean', 'total_weight_sum',
       'total_weight_mean', 'month', 'phone_id', 'month_after_first', 'age',
       'genger_nan', 'gender_male', 'gender_female', 'order_state_complete',
       'order_state_canceled', 'order_state_resumed', 'order_state_cart',
       'shipment_state_shipped', 'shipment_state_canceled',
       'shipment_state_collecting', 'shipment_state_ready',
       'shipment_state_shipping', 'shipment_state_ready_to_ship',
       'shipment_state_pending'],
      dtype='object')

In [10]:
from copy import deepcopy

In [137]:
clf = SGDClassifier()
for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_7, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)
    
    clf.fit(train_features, train_target)
    preds = clf.predict(test_features)
    score = f1_score(test_target, preds)
    print(score)

84876 75399 (75399, 87)
94926 83336 (83336, 87)
0.373938282376823
84519 75071 (75071, 87)
94627 83154 (83154, 87)
0.6644625924189715
84890 75427 (75427, 87)
94896 83168 (83168, 87)
0.6572112546730504


In [94]:
clf.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)>

In [30]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=190, max_depth=14)

param = {'max_depth': 4, 'eta': 1, 'objective': 'binary:logistic'}
for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    #train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    
    #test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

127226 78037 (78037, 31)
142031 78425 (78425, 31)
<class 'xgboost.sklearn.XGBClassifier'>
0.7509349717600367
127059 77545 (77545, 31)
142418 78856 (78856, 31)
<class 'xgboost.sklearn.XGBClassifier'>
0.7529189673887393


IndexError: list index out of range

In [19]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=600, max_depth=5)

param = {'max_depth': 4, 'eta': 1, 'objective': 'binary:logistic'}
for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

84806 75270 (75270, 87)
94927 83380 (83380, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.6975030290917722
84706 75181 (75181, 87)
95015 83309 (83309, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.7059650160866832
84773 75446 (75446, 87)
94507 82969 (82969, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.7014759917792863


In [38]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.05)

for idx in range(2):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

127226 113134 (113134, 87)
142031 124596 (124596, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.7111398311485589
127059 112763 (112763, 87)
142418 125062 (125062, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.7139466170904986


In [24]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=1000, max_depth=5)

param = {'max_depth': 4, 'eta': 1, 'objective': 'binary:logistic'}
for idx in range(2):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

127226 113134 (113134, 87)
142031 124596 (124596, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.7003893645936684
127059 112763 (112763, 87)
142418 125062 (125062, 87)
<class 'xgboost.sklearn.XGBClassifier'>
0.700639577696693


In [51]:
result_clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.05)

test_features = pd.merge(test_data, client_features_567, on='phone_id').fillna(0)

print(len(test_data), len(test_features), test_features.shape)
    
test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['fold_num']
del test_features['phone_id']
del test_features['month_x']
del test_features['month_y']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

result_clf.fit(test_features, test_target)

284449 249658 (249658, 89)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [82]:
subm_features = pd.merge(df_test, client_features_678, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)

In [83]:
len(result_preds)

214609

In [57]:
len(df_test)

214609

In [87]:
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit.csv", index=False)

In [85]:
df_test

,Id,Predicted
0,19843,0.0
1,471287,1.0
2,342522,0.0
3,457799,1.0
4,233778,0.0
...,...,...
214604,16841,1.0
214605,69348,0.0
214606,594974,1.0
214607,320024,0.0


In [ ]:
client_features

In [58]:
len(subm_features)

180172

In [48]:
subm_features.columns

Index(['Id', 'order_id_nunique_x', 'total_cost_sum_x', 'total_cost_min_x',
       'total_cost_max_x', 'total_cost_mean_x', 'total_cost_std_x',
       'rate_mean_x', 'promo_total_sum_x', 'promo_total_mean_x',
       'total_weight_sum_x', 'total_weight_mean_x', 'month_after_first_x',
       'order_state_complete_x', 'order_state_canceled_x',
       'order_state_resumed_x', 'order_state_cart_x',
       'shipment_state_shipped_x', 'shipment_state_canceled_x',
       'shipment_state_collecting_x', 'shipment_state_ready_x',
       'shipment_state_shipping_x', 'shipment_state_ready_to_ship_x',
       'shipment_state_pending_x', 'order_id_nunique_y', 'total_cost_sum_y',
       'total_cost_min_y', 'total_cost_max_y', 'total_cost_mean_y',
       'total_cost_std_y', 'rate_mean_y', 'promo_total_sum_y',
       'promo_total_mean_y', 'total_weight_sum_y', 'total_weight_mean_y',
       'month_after_first_y', 'order_state_complete_y',
       'order_state_canceled_y', 'order_state_resumed_y', 'order_sta

In [ ]:
clf.train()

In [20]:
clf.feature_importances_

array([2.8075944e-03, 4.4088610e-03, 4.6420028e-03, 4.0781121e-03,
       4.2778212e-03, 4.0899110e-03, 4.4004992e-03, 3.8181802e-03,
       5.0168824e-03, 5.6645134e-03, 5.5080703e-03, 1.0146709e-02,
       1.0410837e-02, 3.0625288e-03, 2.3707431e-03, 0.0000000e+00,
       5.1997034e-03, 0.0000000e+00, 1.4342251e-03, 5.4036913e-04,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 4.1685700e-03,
       3.9150976e-03, 5.3464798e-03, 3.9951829e-03, 4.2095720e-03,
       4.6305396e-03, 4.3462738e-03, 3.6234201e-03, 4.4237655e-03,
       5.0196229e-03, 4.5335353e-03, 2.4677131e-02, 7.0485519e-03,
       3.9929179e-03, 1.1314387e-03, 0.0000000e+00, 7.6393946e-03,
       0.0000000e+00, 0.0000000e+00, 1.1349554e-03, 2.6889683e-03,
       2.1559468e-03, 0.0000000e+00, 4.8745956e-02, 3.8003796e-03,
       3.5161895e-03, 3.6679397e-03, 3.9979760e-03, 4.7889454e-03,
       8.9688795e-03, 5.3242957e-03, 4.1669668e-03, 6.8553546e-03,
       9.2443824e-03, 2.2743221e-02, 3.8644876e-03, 3.9030986e

In [121]:
train_features.describe()

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
count,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,7.344400e+04,...,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000,73444.000000
mean,1.606448,218.051935,71.140270,106.280272,90.496485,19.444291,2.121792,-82.053917,-41.375288,2.887928e+04,...,0.072777,0.002941,0.000068,1.326140,0.072777,0.000803,0.000545,0.000041,0.000286,0.000068
std,1.856730,294.667615,80.080667,92.944226,80.415390,36.903454,2.215667,356.369504,180.685895,4.136821e+04,...,0.416070,0.097164,0.008251,2.018045,0.416070,0.040245,0.032164,0.006391,0.023626,0.008251
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-25000.000000,-25000.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,118.000000,98.000000,99.000000,98.000000,0.000000,1.612500,0.000000,0.000000,1.551100e+04,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,316.000000,158.000000,158.000000,158.000000,28.850186,5.000000,0.000000,0.000000,3.948425e+04,...,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,49.000000,7697.000000,1408.000000,1408.000000,1408.000000,770.039285,5.000000,0.000000,0.000000,1.464781e+06,...,33.000000,9.000000,1.000000,111.000000,33.000000,3.000000,2.000000,1.000000,3.000000,1.000000


In [122]:
test_features.describe()

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
count,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,8.316800e+04,...,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000,83168.000000
mean,1.407140,190.886841,62.384162,93.328670,79.365940,17.145588,1.867992,-71.694614,-35.540407,2.541669e+04,...,0.075366,0.002561,0.000024,1.379473,0.075354,0.000818,0.000517,0.000120,0.000301,0.000024
std,1.845483,293.231940,78.384006,94.104048,80.912492,35.426955,2.189694,421.485540,128.279440,4.024700e+04,...,0.492853,0.082965,0.004904,2.137775,0.492770,0.048786,0.033074,0.012974,0.022201,0.004904
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-68467.299805,-10000.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,98.000000,0.000000,98.000000,98.000000,0.000000,0.000000,0.000000,0.000000,1.128850e+04,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,264.000000,108.000000,158.000000,158.000000,11.313708,4.666667,0.000000,0.000000,3.455550e+04,...,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,129.000000,21503.000000,2283.000000,2283.000000,2283.000000,1116.507053,5.000000,0.000000,0.000000,1.994808e+06,...,76.000000,9.000000,1.000000,240.000000,76.000000,6.000000,4.000000,2.000000,2.000000,1.000000


In [18]:
sorted(test_features.columns)

['age',
 'age_x',
 'age_y',
 'gender_female',
 'gender_female_x',
 'gender_female_y',
 'gender_male',
 'gender_male_x',
 'gender_male_y',
 'genger_nan',
 'genger_nan_x',
 'genger_nan_y',
 'month_after_first',
 'month_after_first_x',
 'month_after_first_y',
 'order_id_nunique',
 'order_id_nunique_x',
 'order_id_nunique_y',
 'order_state_canceled',
 'order_state_canceled_x',
 'order_state_canceled_y',
 'order_state_cart',
 'order_state_cart_x',
 'order_state_cart_y',
 'order_state_complete',
 'order_state_complete_x',
 'order_state_complete_y',
 'order_state_resumed',
 'order_state_resumed_x',
 'order_state_resumed_y',
 'promo_total_mean',
 'promo_total_mean_x',
 'promo_total_mean_y',
 'promo_total_sum',
 'promo_total_sum_x',
 'promo_total_sum_y',
 'rate_mean',
 'rate_mean_x',
 'rate_mean_y',
 'shipment_state_canceled',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_collecting',
 'shipment_state_collecting_x',
 'shipment_state_collecting_y',
 'shipment_state

In [111]:
sum(preds)

51772.0

In [109]:
test_target

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
83163    1.0
83164    1.0
83165    0.0
83166    0.0
83167    1.0
Name: target, Length: 83168, dtype: float64

In [105]:
train_features

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,order_state_canceled,order_state_resumed,order_state_cart,shipment_state_shipped,shipment_state_canceled,shipment_state_collecting,shipment_state_ready,shipment_state_shipping,shipment_state_ready_to_ship,shipment_state_pending
0,1.0,98.0,98.0,98.0,98.000000,0.000000,0.0,0.0,0.0,977.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,98.0,98.0,98.0,98.000000,0.000000,0.0,0.0,0.0,3225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,199.0,199.0,199.0,199.000000,0.000000,0.0,0.0,0.0,16695.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,199.0,0.0,199.0,39.800000,88.995506,3.0,0.0,0.0,79830.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73439,2.0,196.0,98.0,98.0,98.000000,0.000000,2.5,0.0,0.0,36321.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73440,2.0,98.0,0.0,98.0,49.000000,69.296465,5.0,0.0,0.0,1000.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
73441,2.0,256.0,98.0,158.0,128.000000,42.426407,5.0,0.0,0.0,23975.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
73442,2.0,418.0,0.0,418.0,209.000000,295.570635,4.5,0.0,0.0,168050.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
train_target

0        0.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
73439    0.0
73440    1.0
73441    1.0
73442    1.0
73443    1.0
Name: target, Length: 73444, dtype: float64

In [92]:
Counter(train_features.columns)

Counter({'order_id_nunique_x': 1,
         'total_cost_sum_x': 1,
         'total_cost_min_x': 1,
         'total_cost_max_x': 1,
         'total_cost_mean_x': 1,
         'total_cost_std_x': 1,
         'rate_mean_x': 1,
         'promo_total_sum_x': 1,
         'promo_total_mean_x': 1,
         'total_weight_sum_x': 1,
         'total_weight_mean_x': 1,
         'month_after_first_x': 1,
         'age_x': 1,
         'genger_nan_x': 1,
         'gender_male_x': 1,
         'gender_female_x': 1,
         'order_state_complete_x': 1,
         'order_state_canceled_x': 1,
         'order_state_resumed_x': 1,
         'order_state_cart_x': 1,
         'shipment_state_shipped_x': 1,
         'shipment_state_canceled_x': 1,
         'shipment_state_collecting_x': 1,
         'shipment_state_ready_x': 1,
         'shipment_state_shipping_x': 1,
         'shipment_state_ready_to_ship_x': 1,
         'shipment_state_pending_x': 1,
         'order_id_nunique_y': 1,
         'total_cost_sum_y':

In [69]:
train_target.values

array([0., 1., 0., ..., 1., 1., 0.])

In [13]:
import xgboost as xgb
from collections import Counter

In [78]:
!pip install xgboost

     |████████████████████████████████| 148.9 MB 234 kB/s eta 0:00:013    |████▊                           | 21.8 MB 14.0 MB/s eta 0:00:10     |██████████████                  | 64.8 MB 8.2 MB/s eta 0:00:11


In [50]:
(set(client_features_567['phone_id'].values)).difference(set(train_fold['phone_id'].values))

{2,
 4,
 6,
 9,
 14,
 16,
 18,
 19,
 23,
 24,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 44,
 45,
 46,
 47,
 48,
 49,
 52,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 68,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 96,
 97,
 99,
 100,
 101,
 102,
 105,
 106,
 109,
 112,
 113,
 114,
 116,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 129,
 131,
 132,
 134,
 135,
 136,
 139,
 140,
 141,
 143,
 144,
 145,
 146,
 147,
 149,
 150,
 151,
 152,
 153,
 155,
 156,
 158,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 171,
 176,
 177,
 178,
 180,
 182,
 183,
 185,
 186,
 187,
 190,
 192,
 193,
 194,
 197,
 198,
 199,
 201,
 202,
 204,
 205,
 206,
 207,
 208,
 210,
 211,
 212,
 213,
 215,
 216,
 218,
 219,
 221,
 223,
 224,
 226,
 228,
 230,
 234,
 235,
 236,
 238,
 240,
 241,
 242,
 243,
 246,
 251,
 252,
 253,
 254,
 259,
 260,
 262,
 263,
 268,
 269,
 271,
 274,
 276,
 277,
 279,
 280,
 281,
 285,
 286,
 288,
 

In [49]:
client_features[client_features['phone_id'].isin(train_fold['phone_id'].values)]['']

4               3
5               3
6               3
18              8
19              8
            ...  
1155554    719113
1155628    719192
1155776    719359
1155777    719360
1155788    719374
Name: phone_id, Length: 258575, dtype: int64

In [44]:
train_fold['phone_id'].nunique()

84890

In [54]:
train_fold

,phone_id,target,month
0,19843,0.0,6
1,173074,0.0,6
2,342522,1.0,6
3,702441,0.0,6
4,40738,1.0,6
...,...,...,...
84885,143151,0.0,6
84886,49252,1.0,6
84887,222552,1.0,6
84888,320024,1.0,6


In [19]:
test_folds

[       phone_id  target  month
 0        173074     0.0      7
 1        101944     0.0      7
 2        615032     0.0      7
 3        174054     1.0      7
 4        155699     1.0      7
 ...         ...     ...    ...
 94921     78431     1.0      7
 94922    652913     1.0      7
 94923    106273     1.0      7
 94924    238680     0.0      7
 94925     69348     0.0      7
 
 [94926 rows x 3 columns],
        phone_id  target  month
 0         19843     1.0      7
 1        457799     1.0      7
 2        233778     1.0      7
 3         31838     0.0      7
 4         57297     1.0      7
 ...         ...     ...    ...
 94622     69813     1.0      7
 94623     66542     1.0      7
 94624    143151     0.0      7
 94625     49252     0.0      7
 94626     64318     0.0      7
 
 [94627 rows x 3 columns],
        phone_id  target  month
 0        342522     0.0      7
 1        702441     0.0      7
 2         40738     0.0      7
 3         86829     0.0      7
 4         685

In [74]:
import lightgbm as lgb